## DM-18736 review

In [2]:
import os
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)

This notebook gives me DIASource and DIAObject dataframes to futz with

In [3]:
# Meredith's standard processing location
#REPO = '/project/mrawls/hits2015/rerun/cw_processed3'
REPO = '/project/mrawls/hits2015/rerun/cw_processed4' #with updated PhotoCalib configs 

# with decorrelation off
#/project/mrawls/hits2015/rerun/cw_nodecorr1

In [4]:
connection = sqlite3.connect(f'{REPO}/association.db')

Take a look at the DiaSource table:

In [15]:
dia_sources = pd.read_sql_query('select * from DiaSource LIMIT 30;', connection)

In [16]:
dia_sources.head()

,diaSourceId,ccdVisitId,diaObjectId,ssObjectId,parentDiaSourceId,prv_procOrder,ssObjectReassocTime,midPointTai,ra,raErr,...,ixxPSF,iyyPSF,ixyPSF,extendedness,spuriousness,flags,pixelId,filterName,filterId,isDipole
0,176486811852341337,41091538,176486811852341337,0,0,0,None,57070.141866,150.908070,None,...,3.568572,4.375425,-0.412995,None,None,268,0,g,1,0
1,176486811852341338,41091538,176486811852341338,0,0,0,None,57070.141866,150.912628,None,...,3.568572,4.375425,-0.412995,None,None,0,0,g,1,0
2,176486811852341339,41091538,176486811852341339,0,0,0,None,57070.141866,150.917767,None,...,3.568572,4.375425,-0.412995,None,None,25165824,0,g,1,1
3,176486811852341340,41091538,176486811852341340,0,0,0,None,57070.141866,150.918168,None,...,3.568572,4.375425,-0.412995,None,None,1320,0,g,1,0
4,176486811852341341,41091538,176486811852341341,0,0,0,None,57070.141866,150.924709,None,...,3.568572,4.375425,-0.412995,None,None,41943900,0,g,1,0


In [17]:
dia_objects = pd.read_sql_query('select * from DiaObject LIMIT 30;', connection)

In [19]:
dia_objects.head()

,diaObjectId,validityStart,validityEnd,ra,raErr,decl,declErr,ra_decl_Cov,radecTai,pmRa,...,yPSFluxPercentile75,yPSFluxPercentile95,yPSFluxMin,yPSFluxMax,yPSFluxStetsonJ,yPSFluxLinearSlope,yPSFluxLinearIntercept,yPSFluxMaxSlope,yPSFluxErrMean,parent
0,176486811852341337,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.908070,None,2.192137,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,0
1,176486811852341338,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.912628,None,2.193383,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,0
2,176486811852341339,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.917767,None,2.109709,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,0
3,176486811852341340,2015-02-17 03:24:17.200177,None,150.918168,None,2.142636,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,0
4,176486811852341341,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.924709,None,2.093516,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,0


In [39]:
currentFluxMask = dia_sources["filterId"] == 1

In [33]:
fluxes = dia_sources.loc[:, "psFlux"].replace([None], np.nan)
fluxErrors = dia_sources.loc[:, "psFluxErr"].replace([None], np.nan)

In [34]:
noNanMask = np.logical_and(np.isfinite(fluxes), np.isfinite(fluxErrors))

In [35]:
noNanMask

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
Name: psFlux, dtype: bool

In [40]:
dia_sources.loc[currentFluxMask & noNanMask, "midPointTai"]

0     57070.141866
1     57070.141866
2     57070.141866
3     57070.141866
4     57070.141866
5     57070.141866
6     57070.141866
7     57070.141866
8     57070.141866
9     57070.141866
10    57070.141866
11    57070.141866
12    57070.141866
13    57070.141866
14    57070.141866
15    57070.141866
16    57070.141866
17    57070.141866
18    57070.141866
19    57070.141866
20    57070.141866
21    57070.141866
22    57070.141866
23    57070.141866
24    57070.141866
25    57070.141866
26    57070.141866
27    57070.141866
28    57070.141866
29    57070.141866
Name: midPointTai, dtype: float64

In [42]:
pd.Series(False,index=dia_objects.index)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
dtype: bool